# Covert Channel Machine Learning

## Loading data and preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
# Location of .obj files in Google Drive
data_location = '/content/drive/MyDrive/CC Machine Learning/'
os.chdir(data_location)

In [ ]:
!ls | grep obj

X_60_active.obj
X_60.obj
X_kernel24.obj
X.obj
X_simple.obj
y_60_active.obj
y_60.obj
y_kernel24.obj
y.obj
y_simple.obj


Now we are in the proper location to access the .obj files

In [ ]:
import pickle
import numpy as np

# wait this is all wrong...

X_file1 = 'X_legacy_tcp.obj'
y_file1 = 'y_legacy_tcp.obj'
X_file2 = 'X_complex_tcp_active.obj'
y_file2 = 'y_complex_tcp_active.obj'

X_files = [X_file1, X_file2]
y_files = [y_file1, y_file2]

standard_length = 15

# initilize 2D array
X = np.empty((0, standard_length), dtype=int)

for X_file in X_files:
  # Load everything like this because pickling buffer has been used
  with open(X_file, 'rb') as f:
    captures_loaded = 0
    while True:
      try:
        new_arr = pickle.load(f)
        if new_arr.size > standard_length:
          new_arr = new_arr[:standard_length]
        elif new_arr.size < standard_length:
          # pad to required length
          while new_arr.size < standard_length:
            new_arr = np.append(new_arr, 0)

        # add to our main 2D array
        X = np.vstack([X, new_arr])
        captures_loaded += 1
      except EOFError:
        break

print(f'Captures loaded: {captures_loaded}')
print(f'X shape: {X.shape}')

y = np.empty(0)

# Load y normally
for y_file in y_files:
  with open(y_file, 'rb') as f:
    while True:
      try:
        y = np.append(y, pickle.load(f))
      except EOFError:
          break

print(f'y shape: {y.shape}')

Captures loaded: 1000
X shape: (2000, 15)
y shape: (2000,)


* X - 2D numpy array (each capture, each packet in that capture)
* y - 1D numpy array for targets (target)

In [ ]:
# Now need to convert y to categorical
from keras.utils import to_categorical
y = to_categorical(y)

In [ ]:
# Print shape information
print(X.shape, '\n', y.shape, sep='')

(2000, 15)
(2000, 2)


In [ ]:
# Generate train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1)

In [ ]:
# Scale data to fit between 0 and 1 range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_train)
scaler.fit(X_test)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X_train)

[[ 0.87938267  0.90783886  0.05558955 ...  0.20100547  0.57346007
   0.64534001]
 [ 0.30060888 -0.00245913  0.20864205 ...  0.59772467  0.82081004
   0.74213449]
 [ 0.63154477  0.63005206  0.62535051 ...  0.64383894  0.6361079
   0.64539097]
 ...
 [ 0.71629959  0.65238817  0.54836402 ...  0.37004796  0.95410326
   0.06737908]
 [ 0.66100919  0.41472135  0.03942056 ...  0.97610338  0.17430053
   0.82907395]
 [ 0.94250476  0.67890327  0.60044464 ...  0.29843863  0.33953331
   0.03865698]]


## Building the network with Keras

In [ ]:
# Import Keras elements needed for the model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import BatchNormalization
from keras.layers import GlobalMaxPooling1D
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import InputLayer
from keras.layers import Add
from keras.layers import LayerNormalization
from keras.layers import MultiHeadAttention
from keras.optimizers import Adam
import tensorflow as tf

In [ ]:
# Define important dimensions and other settings to be used in building the model
# TODO: maybe need to set n_packets to the max amount of packets in one of the captures and then fill 0s for captures with less packets than that
n_isns = X.shape[1]                   # max no. of packets in one capture
n_outputs = y_train.shape[1]          # no. of questions
verbose = 0
epochs = 10
batch_size = 32

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, LSTM, Dense, Bidirectional, InputLayer

# Define the input shape and number of outputs
sequence_length = n_isns  # Replace with your sequence length
n_outputs = 2  # Since you have two classes

# Create the model
model = Sequential()

# Input Layer
model.add(InputLayer(input_shape=(sequence_length, 1)))

# First Conv Block
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Second Conv Block
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))

# Third Conv Block
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# Bidirectional LSTM Layers
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))

# Fully Connected Layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(n_outputs, activation='softmax'))

# Compile the Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the Model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 15, 64)              │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 15, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 15, 64)              │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 15, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 7, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 7, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 7, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 7, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 7, 128)              │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 7, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 3, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 3, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 3, 256)              │         196,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 3, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,018,690 (3.89 MB)

 Trainable params: 1,016,898 (3.88 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [ ]:
# TESTING THE MODEL
import time

repeats = 10
total = 0
total_time = 0
for i in range(repeats):
    print("Test", i+1, "... ", end="")
    start_time = time.time()
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)[1]
    time_elapsed = time.time() - start_time
    print(f"{accuracy*100:.2f}% \t Time: {time_elapsed:.2f}s")
    total += accuracy*100
    total_time += time_elapsed
total_accuracy = total/repeats
print(f"Total: {total_accuracy:.2f}%")
print(f"Average time: {total_time/repeats:.2f}s")


Test 1 ... 100.00% 	 Time: 30.43s
Test 2 ... 100.00% 	 Time: 29.68s
Test 3 ... 100.00% 	 Time: 27.72s
Test 4 ... 100.00% 	 Time: 29.65s
Test 5 ... 100.00% 	 Time: 29.80s
Test 6 ... 100.00% 	 Time: 28.09s
Test 7 ... 100.00% 	 Time: 28.88s
Test 8 ... 100.00% 	 Time: 29.06s
Test 9 ... 100.00% 	 Time: 29.50s
Test 10 ... 100.00% 	 Time: 32.72s
Total: 100.00%
Average time: 29.55s
